In [1]:
# set environment and import packages
import os
from pandas import read_csv
import pandas as pd
import geopandas as gps
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import constants as c
import warnings
import numpy as np

warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync')



In [2]:
sctg_group_lookup = read_csv(c.param_dir + c.sctg_group_lookup_file, sep = ',')
mesozone_lookup = read_csv(c.param_dir + c.mesozone_id_lookup_file, sep = ',')

truck_mode = ['For-hire Truck', 'Private Truck']
# max_ton_lookup = {'sctg1': 218.327, 'sctg2': 51.13091, 'sctg3': 94.56035, 'sctg4': 4.4861, 'sctg5': 51.64694}

In [3]:
combined_modeled_OD = None
mode_choide_by_commodity = None
combined_truck_output = None
for sctg in c.list_of_sctg_group:
    print(sctg)
#     if sctg != 'sctg5':
#         continue
    b2b_dir = c.output_dir + sctg
    list_of_b2b_files = listdir(b2b_dir)
    if sctg == 'sctg4':
        chunk_of_b2b_files = np.array_split(list_of_b2b_files, 50)
    elif sctg == 'sctg5':
        chunk_of_b2b_files = [list_of_b2b_files]
    else:
        chunk_of_b2b_files = np.array_split(list_of_b2b_files, 50)
    iterator = 0
#     cut_off_point = max_ton_lookup[sctg] # shipment capacity for this shipment
    for chunk in chunk_of_b2b_files:
        if len(chunk) == 0:
            continue
#         if file == '.DS_Store':
#             continue
#         if iterator%10 == 0:
        print(iterator)
        filelist = [file for file in chunk if (file.startswith('reassigned') & (file.endswith('.zip')))]
        modeled_OD_by_sctg = pd.concat([read_csv(b2b_dir  + '/' + f, low_memory=False) for f in filelist ])
#         modeled_OD_by_sctg = read_csv(b2b_dir + '/' + file, sep = ',')
#         print(modeled_OD_by_sctg.columns)
        print('data loading finished...')
        print(modeled_OD_by_sctg.TruckLoad.sum()/1000)
        list_of_var = ['BuyerID', 'BuyerZone', 'BuyerNAICS', 'SellerID',
           'SellerZone', 'SellerNAICS', 'TruckLoad', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID', 'mode_choice']
#         print(modeled_OD_by_sctg.head(5))
        
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['BuyerZone'] == 2270, 'BuyerZone'] = 2158
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['SellerZone'] == 2270, 'SellerZone'] = 2158
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['BuyerZone'] == 46113, 'BuyerZone'] = 46102
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['SellerZone'] == 46113, 'SellerZone'] = 46102
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['BuyerZone'] == 51515, 'BuyerZone'] = 51019
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['SellerZone'] == 51515, 'SellerZone'] = 51019
        
        truck_output = modeled_OD_by_sctg.loc[modeled_OD_by_sctg['mode_choice'].isin(truck_mode), list_of_var]
        int_var = ['BuyerID', 'BuyerZone', 'SellerID',
           'SellerZone', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID']
        truck_output.loc[:, int_var] = truck_output.loc[:, int_var].astype(int)
        truck_path = c.output_dir + sctg + '_truck/'
# Check whether the specified path exists or not
        isExist = os.path.exists(truck_path)
        if not isExist:

           # Create a new directory because it does not exist
           os.makedirs(truck_path)
        truck_output.to_csv(truck_path + 'truck_only_OD_' + sctg + '_' + 
                            str(iterator) + '.zip', index = False)

        
        ## compute national shipment count and tonmile
        print('generate summary statistics...')
        modeled_OD_by_sctg['ShipmentLoad'] = modeled_OD_by_sctg['TruckLoad'] / 1000 # convert to thousand tons
        
        #print(cut_off_point)
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['ShipmentLoad'] > cut_off_point, 'ShipmentLoad'] = cut_off_point
        modeled_OD_by_sctg['tmiles'] = modeled_OD_by_sctg['ShipmentLoad'] * 1000 * modeled_OD_by_sctg['Distance']
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['SellerZone', 'orig_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'inner')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "orig_GEOID",
                                                                "CBPZONE": "orig_CBPZONE", 
                                                                "MESOZONE":"orig_MESOZONE", 
                                                                "FAFNAME":"orig_FAFNAME"})
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['BuyerZone', 'dest_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'inner')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "dest_GEOID", 
                                                                "CBPZONE": "dest_CBPZONE", 
                                                                "MESOZONE":"dest_MESOZONE", 
                                                               "FAFNAME":"dest_FAFNAME"})    
        agg_OD_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['tmiles', 'ShipmentLoad']].sum()        
        agg_OD_by_sctg = agg_OD_by_sctg.reset_index()
        agg_count_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['shipment_id']].count() 
        agg_count_by_sctg = agg_count_by_sctg.reset_index()
        agg_OD_by_sctg = pd.merge(agg_OD_by_sctg, agg_count_by_sctg, 
                                  on = ["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'],
                                  how = 'left')
        agg_OD_by_sctg = agg_OD_by_sctg.rename(columns={"shipment_id": "count"})
        agg_OD_by_sctg.loc[:, 'SCTG_Name'] = c.sctg_def[sctg]
        agg_OD_by_sctg.loc[:, 'chunk_id'] = iterator
        print(agg_OD_by_sctg.ShipmentLoad.sum())
        combined_modeled_OD = pd.concat([combined_modeled_OD, agg_OD_by_sctg], sort = False)
        iterator += 1 
#         break        
#     break
#     combined_truck_output.to_csv(c.input_dir + 'truck_only_OD_' + sctg + '.csv', index = False)
# combined_modeled_OD = pd.merge(combined_modeled_OD, sctg_group_definition, on = ['SCTG_Group'], how = 'left')
combined_modeled_OD.head(10)    

sctg1
0
data loading finished...
77813.64035609519
generate summary statistics...
77813.6403560951
1
data loading finished...
93722.86876534708
generate summary statistics...
93722.86876534695
2
data loading finished...
78290.2620598906
generate summary statistics...
78290.26205989067
3
data loading finished...
77789.99319972162
generate summary statistics...
77789.9931997216
4
data loading finished...
86129.29089101494
generate summary statistics...
86129.29089101503
5
data loading finished...
76864.01912866309
generate summary statistics...
76864.01912866312
6
data loading finished...
77770.93474853394
generate summary statistics...
77770.934748534
7
data loading finished...
77749.41436972914
generate summary statistics...
77749.41436972917
8
data loading finished...
85764.98305759224
generate summary statistics...
85764.98305759217
9
data loading finished...
49667.99601649102
generate summary statistics...
49667.99601649108
10
data loading finished...
85324.83203940598
generate summ

,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,mode_choice,tmiles,ShipmentLoad,count,SCTG_Name,chunk_id
0,11,Birmingham,139,Rest of GA,1,For-hire Truck,4.566379e+06,14.868871,578,bulk,0
1,11,Birmingham,139,Rest of GA,1,Private Truck,2.685946e+05,0.874588,34,bulk,0
2,11,Birmingham,139,Rest of GA,1,Rail/IMX,5.219716e+05,1.235438,48,bulk,0
3,11,Birmingham,190,Iowa,1,For-hire Truck,3.892195e+06,3.695484,144,bulk,0
4,11,Birmingham,190,Iowa,1,Rail/IMX,2.574005e+06,2.309678,90,bulk,0
5,11,Birmingham,219,Rest of KY,1,For-hire Truck,4.523511e+06,12.538798,488,bulk,0
6,11,Birmingham,219,Rest of KY,1,Private Truck,2.410067e+05,0.668051,26,bulk,0
7,11,Birmingham,219,Rest of KY,1,Rail/IMX,7.368130e+05,1.490267,58,bulk,0
8,11,Birmingham,221,Baton Rouge,1,For-hire Truck,1.118468e+07,13.469193,524,bulk,0
9,11,Birmingham,221,Baton Rouge,1,Rail/IMX,2.987513e+06,4.035617,157,bulk,0


In [4]:
combined_modeled_OD_agg = combined_modeled_OD.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", \
                                                       "dest_FAFNAME", "SCTG_Group", 'SCTG_Name',
                                                       'mode_choice'])[['tmiles', 'ShipmentLoad', 'count']].sum()
combined_modeled_OD_agg = combined_modeled_OD_agg.reset_index()
combined_modeled_OD_agg.head(5)
# combined_modeled_OD_agg.loc[:, 'in_study_area'] = 0
# buffer = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.bay_area_region_code) | \
#         combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.bay_area_region_code)
# combined_modeled_OD_agg.loc[buffer, 'in_study_area'] = 1

# combined_modeled_OD_agg.loc[:, 'outbound'] = 0
# combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.region_code), 'outbound'] = 1

# combined_modeled_OD_agg.loc[:, 'inbound'] = 0
# combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.region_code), 'inbound'] = 1

combined_modeled_OD_agg.loc[:, 'orig_FAFID'] = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'dest_FAFID'] = combined_modeled_OD_agg.loc[:, 'dest_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'SCTG_Group'] = combined_modeled_OD_agg.loc[:, 'SCTG_Group'].astype(int)
combined_modeled_OD_agg.loc[:, 'Distance'] = combined_modeled_OD_agg.loc[:, 'tmiles'] / 1000 / combined_modeled_OD_agg.loc[:, 'ShipmentLoad']
combined_modeled_OD_agg.head(10)

,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,SCTG_Name,mode_choice,tmiles,ShipmentLoad,count,Distance
0,11,Birmingham,11,Birmingham,1,bulk,For-hire Truck,0.0,6356.016896,162598,0.0
1,11,Birmingham,11,Birmingham,1,bulk,Private Truck,0.0,3783.134223,93422,0.0
2,11,Birmingham,11,Birmingham,1,bulk,Rail/IMX,0.0,242.700011,5760,0.0
3,11,Birmingham,11,Birmingham,2,fuel_fert,Air,0.0,0.058062,25,0.0
4,11,Birmingham,11,Birmingham,2,fuel_fert,For-hire Truck,0.0,639.902371,38773,0.0
5,11,Birmingham,11,Birmingham,2,fuel_fert,Parcel,0.0,0.000010,1,0.0
6,11,Birmingham,11,Birmingham,2,fuel_fert,Private Truck,0.0,703.734493,50206,0.0
7,11,Birmingham,11,Birmingham,2,fuel_fert,Rail/IMX,0.0,27.630723,1060,0.0
8,11,Birmingham,11,Birmingham,3,interm_food,Air,0.0,0.034425,23,0.0
9,11,Birmingham,11,Birmingham,3,interm_food,For-hire Truck,0.0,271.657661,18797,0.0


In [5]:
combined_modeled_OD_agg.to_csv(c.output_dir + 'processed_b2b_flow_summary.csv', sep = ',')